In [ ]:
# TODO REFACTOR: Integrate code from _legacy/inference.py into this notebook

## This demo app shows how to query Llama 2 using the Gradio UI.

Since we are using Replicate in this example, you will need to replace `<your replicate api token>` with your API token.

To get the Replicate token: 

- You will need to first sign in with Replicate with your github account
- Then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while 

**Note** After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

To run this example:
- Set up your Replicate API token and enter it in place of `<your replicate api token>`
- Run the notebook
- Enter your question and click Submit

In the notebook or a browser with URL http://127.0.0.1:7860 you should see a UI with your answer.

In [2]:
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
from langchain.llms import Replicate
import os

os.environ["REPLICATE_API_TOKEN"] = "<your replicate api token>"

llama2_13b_chat = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)


def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(message) #history_langchain_format)
    return gpt_response#.content

gr.ChatInterface(predict).launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://da04ca953d96e094f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**Next**, we are going to use the `transformers` library for loading and interacting with the model.

Loading the Local Model:
- Used transformers library to load the local model and tokenizer.

Model Inference:
- Modified the predict function to prepare the input text for the model.
- Generated the response using the local model.

In [4]:
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to your local model directory
local_model_path = "/data/ai/models/nlp/llama/models_llama2/llama-2-7b-chat-hf"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Set the model to evaluation mode
model.eval()

def predict(message, history):
    # Convert the history to the required format
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    
    # Prepare the input for the model
    input_text = " ".join([msg.content for msg in history_langchain_format])
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate the response using the model
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.01, top_p=1)

    # Decode the generated response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

gr.ChatInterface(predict).launch(share=True)

Loading checkpoint shards: 100%|██████████| 2/2 [05:34<00:00, 167.03s/it]


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://777539071df159b443.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
